## 定期預金の契約についてのデータセット

データの取得
- https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip<br>
bank.csv を使用

データの説明
- https://archive.ics.uci.edu/ml/datasets/bank+marketing

内容を確認
- "age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"<br>
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
- "y" 定期預金を預けてくれたかどうか

In [ ]:
import pandas as pd
import numpy as np

bank = pd.read_csv('data/bank.csv', sep=';')

# 使用する列のみからなる DataFrame を作成
bank = bank[['y', 'job', 'campaign']]

# 'y'列のダミー変数化: yes->1, no->0
bank['y'] = bank['y'].map(lambda x: 1 if x == 'yes' else 0)

# 列ごとの欠損値の数
bank.isna().sum()

In [ ]:
bank.head(2)

- y: 定期預金を契約したか（目的変数）
- job: 職種
- campaign: 勧誘のコンタクト回数

<hr>

### 授業参加確認問題

2. job列の retired, student, unemployed, unknown は無職、それ以外を有職とした時、有職者の数を答えない。

In [ ]:
# 有職者でなければ 0 を返す関数を定義
def employed(x):
    if x in ['retired', 'student', 'unemployed', 'unknown']:
        return 0
    return 1

# 'job'列のダミー変数化
bank['employed'] = bank['job'].map(lambda x: employed(x))

# 確認
bank.employed.value_counts()

employed が 1 となるのが有職者数

単純に for ループでカウントすることも可能：

In [ ]:
num_employed = 0

for j in bank['job']:
    if j != 'retired' and \
       j != 'student' and \
       j != 'unemployed' and \
       j != 'unknown':
        num_employed = num_employed + 1
        
print(num_employed)

value_counts() で職種ごとに集計した後に計算することも可能：

In [ ]:
counts = bank.value_counts('job')
len(bank) - counts[['retired', 'student', 'unemployed', 'unknown']].sum()

3. job列は上記の場合分けで有職 (employed=1), 無職 (employed=0)とするダミー変数化を行った employed 列を追加し、job列は削除する。そのうえで、y を目的変数、campaign, employed を説明変数として、ロジスティック回帰モデルを作成し、AUCを算出しなさい。<br>
・小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.1234ならば0.123が答え）。

- employed 列は追加済み。

In [ ]:
# 'job' 列を削除
bank = bank.drop(['job'], axis=1)

# 確認
bank

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

def Logistic_Model(X, Y):
    """
    ロジスティック回帰モデルを作成
    """
    model = LogisticRegression(C=10000.0)
    model.fit(X, Y)
    return model

def calc_AUC(model, X, Y):
    """
    AUCの算出
    """
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    return auc(fpr, tpr)

In [ ]:
# 説明変数、目的変数を作成
Y_label = 'y'
X = bank.drop([Y_label], axis=1)
Y = bank[Y_label]

model = Logistic_Model(X, Y)
print('AUC={:.3f}'.format(calc_AUC(model, X, Y)))

4. y を目的変数、campaign のみを説明変数として作成したロジスティック回帰モデルを分析し、勧誘のコンタクト回数が増えると契約がとれる確率は上がるかどうか答えなさい。

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

# 散布図にシグモイド曲線を重ねて表示
def sigmoid(x, b0, b1):
    return 1/(1 + np.exp(-(b0 + b1 * x)))

def Logistic_Plot(model, X, Y):
    # 散布図
    sns.scatterplot(x=X.iloc[:,0], y=Y)

    # パラメータ
    b0 = model.intercept_[0]
    b1 = model.coef_[0][0]

    # シグモイド曲線
    _max = X.iloc[:,0].max()
    _min = X.iloc[:,0].min()
    _step = (_max - _min) / 100
    _X = np.arange(_min, _max + _step, _step)
    plt.plot(_X, sigmoid(_X, b0, b1), color='brown')
    
    # パラメータの表示
    print('b0 = {:.3f}, b1 ={:.3f}'.format(b0, b1))

In [ ]:
# 説明変数、目的変数を作成
X = bank[['campaign']]
Y = bank['y']

model = Logistic_Model(X, Y)
Logistic_Plot(model, X, Y)

勧誘のコンタクト回数 campaign を増やしても、契約がとれる確率 y は上がらない

回帰係数 (b1) が負になっていることからも裏付けられる。

<hr>

### 質問

問3の結果からわかることは何か？

- AUC=0.579
- 混合行列では全てが陰性（定期預金を契約しない）の予測
```
array([[4000,    0],
       [ 521,    0]])
```

ロジスティック回帰もモデルから得られた AUC=0.579 というランダムに近いAUC値からわかることは、ロジスティック回帰では「campaignと有職者かどうかというデータからは、定期預金を預けてくれるかどうかは分からない」ということ。

定期預金を預けてくれる人を探すには

- ロジスティック回帰では「別の説明変数を用いたモデルを作成してみた方が良い」
- 他の分析手法（決定木分析など）を用いてデータの理解をすすめる、重要な説明変数を探す

ということになる。

### 質問

LogisticRegressionの正則化パラメータCを1や0.01の値に変えても、AUC値にはほとんど変化はなかったが、パラメータを変えて結果的にAUCの値が大きくなれば良いということか？

正則化により外れ値の影響を減らすという意味ではそうなる。

### 質問

```
bank.loc[bank["job"]== 'management', "employeed"]=1
bank.loc[bank["job"]== 'retired', "employeed"]=0
```
という形で新しいデータフレームを作成し、7_4_Defaultを参考にロジスティック回帰モデルを作成したところ、右下に下がっていくかたちのシグモイド曲線になったが、それはおかしくないか？

X軸の値が増えると確率が下がるということを表しているため、問題ない。